In [1]:
import cv2
import mediapipe as mp
import time
import numpy as np
from firebase import firebase
from keras.preprocessing import image
firebase= firebase.FirebaseApplication("https://finalppt-a818d-default-rtdb.firebaseio.com/", None)
font = cv2.FONT_HERSHEY_SIMPLEX
org = (50, 50)
fontScale = 1
color = (255, 0, 0)
thickness = 2
from tensorflow import keras
cap = cv2.VideoCapture(0)

mpHands = mp.solutions.hands
hands = mpHands.Hands(static_image_mode=False,
                      max_num_hands=1,
                      min_detection_confidence=0.5,
                      min_tracking_confidence=0.5)
mpDraw = mp.solutions.drawing_utils

pTime = 0
cTime = 0
while True:
    success, img = cap.read()
    img = cv2.flip(img,1)
    lx=[]
    ly=[]
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    #print(results.multi_hand_landmarks)
    if results.multi_hand_landmarks:
        
        for handLms in results.multi_hand_landmarks:
            for id, lm in enumerate(handLms.landmark):
                #print(id,lm)
                h, w, c = img.shape
                cx, cy = int(lm.x *w), int(lm.y*h)
                lx.append(cx)
                ly.append(cy)
                cv2.circle(img, (cx,cy), 3, (255,0,255), cv2.FILLED)
        mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
        #cv2.rectangle(img, (min(lx)-50,min(ly)-50), (max(lx)+50,max(ly)+50), (255, 0, 0), 2)
        if ly[8]== min(ly):
            gesture= "isJump"
        elif ly[6]<ly[8] and ly[10]<ly[12] and ly[14]<ly[16] and ly[18]<ly[20]:
            gesture= "isHold"
        elif ly[6]>ly[8] and ly[10]>ly[12] and ly[14]>ly[16] and ly[18]>ly[20]:
            gesture= "isMove"
    else:
        gesture= "isHold"
            
    img = cv2.putText(img, gesture , org, font, 
                       fontScale, color, thickness, cv2.LINE_AA)
    my_file = open("intermediate_script.txt", "w")
    my_file.write(gesture)
    img= cv2.resize(img,(720,720))

    cv2.imshow("Image", img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# After the loop release the cap object
cap.release()
# Destroy all the windows
cv2.destroyAllWindows()
